<a href="https://colab.research.google.com/github/3adel/auto-correct/blob/main/training/t5_text_spelling_local_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install transformers

In [2]:
#Setup logging

#from google.colab import drive
import datetime
import logging

batch_size = 16
num_epochs = 5
finetuning_iteration = "8"


#drive.mount('/content/drive/')

# Function to write logs with timestamp
now_log_file_name_suffix = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
log_file_path = f'/Users/adel/adel/dev/playground/auto-correct/logs/{now_log_file_name_suffix}-t5-text_spelling_trainer_v{finetuning_iteration}.log'
def log_message(message):
    # Get the current date and time
    now = datetime.datetime.now()
    # Format the current date and time as a string
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
    # Prepare the log entry
    log_entry = f"{timestamp} - {message}\n"
    # Open the log file and append the log entry
    with open(log_file_path, 'a') as file:
        file.write(log_entry)

# Example usage
log_message("Execution starts")

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import random

# Load the pre-trained T5-small model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)

/Users/adel/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Define the task-specific head
task_prefix = "correct: "


In [5]:
import pandas as pd

# URL pointing directly to the raw CSV data
url = 'https://raw.githubusercontent.com/3adel/auto-correct/main/training_sets_dumb/frequency-alpha-alldicts_1000_words.csv'

try:
    # Attempt to read the CSV file from the provided URL
    data = pd.read_csv(url)
    print("Data loaded successfully!")
except Exception as e:
    # Print the error if the file cannot be read
    print(f"An error occurred: {e}")

training_data = list(zip(data['misspelling'], data['correct_word']))


Data loaded successfully!


In [136]:
# Now text_pairs contains tuples in the format (incorrect, correct)
print("# rows in training data touples = ",len(training_data))
log_message(f"# rows in training data = {len(training_data)}")
log_message(f"############ Random seen data to be validated by the model later.")

seen_validation_data = []
number_seen_data_points = 100

for i in range (number_seen_data_points):
  random_range_start_index = random.randrange(1,9050,1)
  for row in training_data[random_range_start_index:random_range_start_index+1]:
    #print(row[0])
    seen_validation_data.append(row)
print(f"Seen data: {seen_validation_data}")
log_message(f"Seen data: {seen_validation_data}")


# rows in training data touples =  10223
Seen data: [(' tecnical', 'technical'), (' scienct', 'science'), (' agrement', 'agreement'), (' carachter', 'character'), (' reffered', 'referred'), (' navir', 'never'), ('dutie', 'duty'), ('flwo', 'flow'), (' finill', 'final'), (' innyns', 'in'), (' nutural', 'natural'), (' sehie', 'see'), (' tecnology', 'technology'), (' volumee', 'volume'), (' defindant', 'defendant'), (' kpep', 'keep'), (' Wesrt', 'West'), (' regiin', 'region'), ('capacitiy', 'capacity'), (' dayz', 'days'), (' feildling', 'field'), (' kewe', 'knew'), (' maede', 'made'), (' devoloped', 'developed'), (' basik', 'basic'), (' timse', 'times'), ('fijure', 'Figure'), (' stnadarad', 'standard'), (' soin', 'son'), (' buisnesss', 'business'), (' begnning', 'beginning'), (' cease', 'case'), (' eniuuq', 'enough'), ('baasis', 'basis'), (' refured', 'referred'), ('judgiment', 'judgment'), (' reccent', 'recent'), (' Devision', 'Division'), ('specail', 'special'), (' meenz', 'means'), (' t

In [7]:
#Load unseen data by the model
unseen_validation_data = [
    ("Appel", "Apple"),
    ("Booke", "Book"),
    ("Kare", "Car"),
    ("Dög", "Dog"),
    ("Hoise", "House"),
    ("Trei", "Tree"),
    ("Freind", "Friend"),
    ("Hapy", "Happy"),
    ("Bieg", "Big"),
    ("Littel", "Little"),
    ("Rune", "Run"),
    ("Jummp", "Jump"),
    ("Sée", "See"),
    ("Heer", "Hear"),
    ("Saiy", "Say"),
    ("Goe", "Go"),
    ("Comey", "Come"),
    ("Stoip", "Stop"),
    ("Starit", "Start"),
    ("Fynish", "Finish"),
    ("Wont", "Want"),
    ("Neid", "Need"),
    ("Likie", "Like"),
    ("Lovve", "Love"),
    ("Heelp", "Help"),
    ("Tayk", "Take"),
    ("Givv", "Give"),
    ("Mekke", "Make"),
    ("Doe", "Do"),
    ("Noow", "Know"),
    ("Thinck", "Think"),
    ("Fiel", "Feel"),
    ("Sée", "See"),
    ("Heer", "Hear"),
    ("Seee", "See"),
    ("Heer", "Hear"),
    ("Eit", "Eat"),
    ("Drinik", "Drink"),
    ("Sleeep", "Sleep"),
    ("Wakie", "Wake"),
    ("Wurk", "Work"),
    ("Plaiy", "Play"),
    ("Lernn", "Learn"),
    ("Readd", "Read"),
    ("Rrite", "Write"),
    ("Tawk", "Talk"),
    ("Lissten", "Listen"),
    ("Smilie", "Smile"),
    ("Laught", "Laugh"),
    ("Crie", "Cry"),
    ("Yelle", "Yell"),
    ("Hurrie", "Hurry"),
    ("Waite", "Wait"),
    ("Hoipe", "Hope"),
    ("Wisshe", "Wish"),
    ("Trii", "Try"),
    ("Fale", "Fail"),
    ("Suceede", "Succeed"),
    ("Begn", "Begin"),
    ("Ennd", "End"),
    ("Mornig", "Morning"),
    ("Afteernoon", "Afternoon"),
    ("Eveninng", "Evening"),
    ("Nite", "Night"),
    ("Dei", "Day"),
    ("Wieck", "Week"),
    ("Montn", "Month"),
    ("Yeer", "Year"),
    ("Centurie", "Century"),
    ("Lief", "Life"),
    ("Tymme", "Time"),
    ("Spase", "Space"),
    ("Energee", "Energy"),
    ("Watair", "Water"),
    ("Firre", "Fire"),
    ("Aire", "Air"),
    ("Erth", "Earth"),
    ("Skye", "Sky"),
    ("Sone", "Sun"),
    ("Munne", "Moon"),
    ("Phenomonal", "Phenomenal"),
    ("Compitition", "Competition"),
    ("Independacently", "Independently"),
    ("Unrelevent", "Irrelevant"),
    ("Disapointment", "Disappointment"),
    ("Requirments", "Requirements"),
    ("Impossiblity", "Impossibility"),
    ("Exsceptional", "Exceptional"),
    ("Incurrable", "Incurred"),
    ("Unnoticable", "Unnoticeable"),
    ("Misunderstnding", "Misunderstanding"),
    ("Reccomendation", "Recommendation"),
    ("Incompatiblity", "Incompatibility"),
    ("Exsistential", "Existential"),
    ("Irrelevent", "Irrelevant"),
    ("Unpredicatable", "Unpredictable"),
    ("Disastorous", "Disastrous"),
    ("Accomplishement", "Accomplishment"),
    ("Inefficiency", "Inefficacy"),
    ("Impracticality", "Impracticability")
]

log_message(f"############ Unseen data to be validated by the model later.")
print(f"Number of rows in unseen validation data = {len(unseen_validation_data)}")
log_message(f"Number of rows in unseen validation data = {len(unseen_validation_data)}")


print(f"Unseen validation data: {unseen_validation_data}")
log_message(f"Unawwn validation data: {unseen_validation_data}")



Number of rows in unseen validation data = 100
Unseen validation data: [('Appel', 'Apple'), ('Booke', 'Book'), ('Kare', 'Car'), ('Dög', 'Dog'), ('Hoise', 'House'), ('Trei', 'Tree'), ('Freind', 'Friend'), ('Hapy', 'Happy'), ('Bieg', 'Big'), ('Littel', 'Little'), ('Rune', 'Run'), ('Jummp', 'Jump'), ('Sée', 'See'), ('Heer', 'Hear'), ('Saiy', 'Say'), ('Goe', 'Go'), ('Comey', 'Come'), ('Stoip', 'Stop'), ('Starit', 'Start'), ('Fynish', 'Finish'), ('Wont', 'Want'), ('Neid', 'Need'), ('Likie', 'Like'), ('Lovve', 'Love'), ('Heelp', 'Help'), ('Tayk', 'Take'), ('Givv', 'Give'), ('Mekke', 'Make'), ('Doe', 'Do'), ('Noow', 'Know'), ('Thinck', 'Think'), ('Fiel', 'Feel'), ('Sée', 'See'), ('Heer', 'Hear'), ('Seee', 'See'), ('Heer', 'Hear'), ('Eit', 'Eat'), ('Drinik', 'Drink'), ('Sleeep', 'Sleep'), ('Wakie', 'Wake'), ('Wurk', 'Work'), ('Plaiy', 'Play'), ('Lernn', 'Learn'), ('Readd', 'Read'), ('Rrite', 'Write'), ('Tawk', 'Talk'), ('Lissten', 'Listen'), ('Smilie', 'Smile'), ('Laught', 'Laugh'), ('Crie', '

In [8]:
# Preprocess the data
def preprocess_data(input_texts, target_texts):

    #To address the issue of potential non-string types and None values in your input_texts and target_texts
    inputs = [task_prefix + str(text) for text in input_texts if text is not None]
    target_texts = [str(text) for text in target_texts if text is not None]

    # Tokenize the inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(target_texts, max_length=512, truncation=True, padding="max_length", return_tensors="pt")["input_ids"]
    return model_inputs, labels

# Prepare the training data
input_texts, target_texts = zip(*training_data)
train_inputs, train_labels = preprocess_data(input_texts, target_texts)

In [9]:
# Inference method
def correct_text(input_text):
    input_ids = tokenizer.encode(task_prefix + input_text, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=512)[0]
    corrected_text = tokenizer.decode(output_ids, skip_special_tokens=True)
    return corrected_text

In [10]:
#run inference BEFORE finetuning against SEEN data
%%time
log_message(f"############ run inference BEFORE finetuning against SEEN data")
for i, row in enumerate(seen_validation_data):
  corrected_text = correct_text(row[0])
  print(f"{i}:{row[0]}--->{corrected_text}")
  log_message(f"{i}:{row[0]}--->{corrected_text}")



0: givie--->: givie
1: dattae--->correct: dattae
2:moirng--->correct: moirng
3: potentional--->Correct: potentional
4: dutu--->correct: dutu
5: Cenetr--->correct correct: Cenetr
6: indusrrial--->correct:
7: kownen--->correct: kownen
8:knowledg--->correct: knowledg
9: beynod--->correct: beynod
10: prioor--->correct: prioor
11: considir--->correct: considir
12: saund--->correct: saund
13: resntt--->resntt
14: olldo--->correct: olldo
15:subjict--->correct: subjict
16: lisse--->lisse
17: paiid--->correct: paiid
18: heare--->: heare
19: perfromence--->Perfromence
20: plasee--->: plasee
21: lols--->-
22: tiems--->
23: greiter--->greiter
24: movemnt--->correct: movemnt
25: tookei--->correct: tookei
26: Cherimann--->correct: Cherimann
27: namme--->correct: namme
28: sttaef--->sttaef
29: courttt--->correct: courttt
30: posibl--->correct: posibl
31:capitall--->: capitall
32: earthe--->: earthe
33: Direktor--->Correct: Direktor
34: Yrook--->Yrook
35: tigther--->correct:
36: Councel--->correct cor

In [11]:
#run inference BEFORE finetuning against UNSEEN data
%%time
log_message(f"############ run inference BEFORE finetuning against UNSEEN data")
for i, row in enumerate(unseen_validation_data):
  corrected_text = correct_text(row[0])
  print(f"{i}:{row[0]}--->{corrected_text}")
  log_message(f"{i}:{row[0]}--->{corrected_text}")

0:Appel--->Appel
1:Booke--->Booke
2:Kare--->Kare
3:Dög--->Dög: Dög Dög: Dög Dög: Dög Dög: Dög: Dög: Dög: Dög: Dög: Dög
4:Hoise--->Hoise
5:Trei--->correct: Trei
6:Freind--->Freind: Freind
7:Hapy--->Hapy: Hapy
8:Bieg--->Bieg
9:Littel--->Littel Littel Littel: Littel Littel Littel Littel Littel: Littel
10:Rune--->Rune
11:Jummp--->Jummp: Jummp Jummp: Jummp Jummp Jummp Jummp: Jummp: Jummp: Jummp Jummp: Jummp
12:Sée--->Sé
13:Heer--->Heer: Heer: Heer: Heer Heer: Heer
14:Saiy--->Saiy
15:Goe--->Goe: Goe: Goe: Goe Goe: Goe Goe: Goe: Goe: Goe: Goe: Goe: Goe
16:Comey--->Comey
17:Stoip--->Stoip
18:Starit--->Starit: Starit
19:Fynish--->Fynish: Fynish: Fynish: Fynish Fynish: Fynish
20:Wont--->Wont
21:Neid--->correct: Neid
22:Likie--->Likie
23:Lovve--->: Lovve
24:Heelp--->Heelp
25:Tayk--->Tayk
26:Givv--->Givv
27:Mekke--->Mekke
28:Doe--->Doe
29:Noow--->Noow
30:Thinck--->: Thinck
31:Fiel--->Fiel
32:Sée--->Sé
33:Heer--->Heer: Heer: Heer: Heer Heer: Heer
34:Seee--->Seee
35:Heer--->Heer: Heer: Heer: Heer He

In [12]:
#fine tuning w/ batch procerssing
from torch.utils.data import DataLoader, TensorDataset

# Create a TensorDataset
dataset = TensorDataset(train_inputs["input_ids"], train_inputs["attention_mask"], train_labels)

# Create a DataLoader
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print("# training rows per batch:", len(train_dataloader))
log_message(f"# training rows per batch = {len(train_dataloader)}")



# training rows per batch: 639


In [13]:
# Fine-tune the model w/ frequent output
#%%time
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(num_epochs):
    total_loss = 0
    current_batch = 0

    for batch in train_dataloader:
        batch_start_time = datetime.datetime.now()
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} Started batch #{current_batch+1}/{len(train_dataloader)} in epoch #{epoch+1}/{num_epochs}")
        log_message(f"Started batch #{current_batch+1}/{len(train_dataloader)} in epoch #{epoch+1}/{num_epochs}")

        # Unpack the batch
        batch_input_ids, batch_attention_mask, batch_labels = batch


        # Zero the gradients on each iteration
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)


        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print progress
        batch_end_time = datetime.datetime.now()
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} Finished batch #{current_batch+1}/{len(train_dataloader)} in epoch #{epoch+1}/{num_epochs} in ({(batch_end_time - batch_start_time).total_seconds():.2f} seconds)")
        log_message(f"Finished batch #{current_batch+1}/{len(train_dataloader)} in epoch #{epoch+1}/{num_epochs}  in ({(batch_end_time - batch_start_time).total_seconds():.2f} seconds)")

        current_batch += 1



    # Calculate average loss over the number of batches
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
    log_message(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

Streaming output truncated to the last 5000 lines.
2024-05-01 00:51:48 Started batch #59/639 in epoch #2/5
2024-05-01 00:52:06 Finished batch #59/639 in epoch #2/5 in (17.38 seconds)
2024-05-01 00:52:06 Started batch #60/639 in epoch #2/5
2024-05-01 00:52:24 Finished batch #60/639 in epoch #2/5 in (17.98 seconds)
2024-05-01 00:52:24 Started batch #61/639 in epoch #2/5
2024-05-01 00:52:41 Finished batch #61/639 in epoch #2/5 in (17.05 seconds)
2024-05-01 00:52:41 Started batch #62/639 in epoch #2/5
2024-05-01 00:52:58 Finished batch #62/639 in epoch #2/5 in (17.56 seconds)
2024-05-01 00:52:58 Started batch #63/639 in epoch #2/5
2024-05-01 00:53:16 Finished batch #63/639 in epoch #2/5 in (17.83 seconds)
2024-05-01 00:53:16 Started batch #64/639 in epoch #2/5
2024-05-01 00:53:34 Finished batch #64/639 in epoch #2/5 in (18.00 seconds)
2024-05-01 00:53:34 Started batch #65/639 in epoch #2/5
2024-05-01 00:53:52 Finished batch #65/639 in epoch #2/5 in (17.62 seconds)
2024-05-01 00:53:52 Start

In [14]:
#run inference AFTER finetuning against SEEN data
%%time
log_message(f"############ run inference AFTER finetuning against SEEN data")
for i, row in enumerate(seen_validation_data):
  corrected_text = correct_text(row[0])
  print(f"{i}:{row[0]}--->{corrected_text}")
  log_message(f"{i}:{row[0]}--->{corrected_text}")

0: givie--->public
1: dattae--->used
2:moirng--->right
3: potentional--->power
4: dutu--->you
5: Cenetr--->Ce
6: indusrrial--->industry
7: kownen--->kow
8:knowledg--->knowledge
9: beynod--->be
10: prioor--->prior
11: considir--->cons
12: saund--->a
13: resntt--->re
14: olldo--->o
15:subjict--->sub
16: lisse--->light
17: paiid--->pay
18: heare--->hear
19: perfromence--->per
20: plasee--->place
21: lols--->too
22: tiems--->tie
23: greiter--->gre
24: movemnt--->move
25: tookei--->took
26: Cherimann--->Cher
27: namme--->nam
28: sttaef--->s
29: courttt--->court
30: posibl--->work
31:capitall--->capital
32: earthe--->earth
33: Direktor--->Direct
34: Yrook--->year
35: tigther--->side
36: Councel--->C
37: Jamez--->Jam
38: exchagne--->ex
39: secuitry--->history
40: finnally--->end
41:Nortr--->Nor
42: hius--->hi
43: havvee--->having
44: knowleage--->knowledge
45: wqanted--->given
46: lkikle--->year
47: althoug--->alt
48: parets--->pare
49: mothre--->mo
50: Liist--->Li
51: assotiated--->as
52: ac

In [15]:
#run inference AFTER finetuning against UNSEEN data
%%time
log_message(f"############ run inference AFTER finetuning against UNSEEN data")
for i, row in enumerate(unseen_validation_data):
  corrected_text = correct_text(row[0])
  print(f"{i}:{row[0]}--->{corrected_text}")
  log_message(f"{i}:{row[0]}--->{corrected_text}")

0:Appel--->Appel
1:Booke--->Book
2:Kare--->Kar
3:Dög--->Dö
4:Hoise--->host
5:Trei--->Tre
6:Freind--->Free
7:Hapy--->Ha
8:Bieg--->Bi
9:Littel--->Lit
10:Rune--->run
11:Jummp--->government
12:Sée--->S
13:Heer--->He
14:Saiy--->Sa
15:Goe--->Go
16:Comey--->Come
17:Stoip--->ground
18:Starit--->Star
19:Fynish--->F
20:Wont--->place
21:Neid--->Ne
22:Likie--->Li
23:Lovve--->a
24:Heelp--->He
25:Tayk--->Ta
26:Givv--->Gi
27:Mekke--->Me
28:Doe--->Do
29:Noow--->No
30:Thinck--->Thi
31:Fiel--->Fi
32:Sée--->S
33:Heer--->He
34:Seee--->See
35:Heer--->He
36:Eit--->interest
37:Drinik--->English
38:Sleeep--->S
39:Wakie--->first
40:Wurk--->a
41:Plaiy--->Pla
42:Lernn--->learning
43:Readd--->read
44:Rrite--->line
45:Tawk--->Ta
46:Lissten--->Li
47:Smilie--->S
48:Laught--->L
49:Crie--->Cri
50:Yelle--->English
51:Hurrie--->Hur
52:Waite--->waited
53:Hoipe--->person
54:Wisshe--->Wi
55:Trii--->Tri
56:Fale--->Fal
57:Suceede--->Su
58:Begn--->starting
59:Ennd--->En
60:Mornig--->Mor
61:Afteernoon--->North
62:Eveninng--->e

In [16]:
# Save the model and tokenizer
#Save model to Local Drive
model.save_pretrained(f"/Users/adel/adel/dev/playground/auto-correct/models/{finetuning_iteration}")
tokenizer.save_pretrained(f"/Users/adel/adel/dev/playground/auto-correct/models/{finetuning_iteration}")


('/Users/adel/adel/dev/playground/auto-correct/models/8/tokenizer_config.json',
 '/Users/adel/adel/dev/playground/auto-correct/models/8/special_tokens_map.json',
 '/Users/adel/adel/dev/playground/auto-correct/models/8/spiece.model',
 '/Users/adel/adel/dev/playground/auto-correct/models/8/added_tokens.json')

In [17]:
# How to load the model from drive

finetuned_model_path = f"/Users/adel/adel/dev/playground/auto-correct/models/{finetuning_iteration}"
finetuned_tokenizer = T5Tokenizer.from_pretrained(finetuned_model_path)
model_finetuned_spelling_en_1000_words = T5ForConditionalGeneration.from_pretrained(finetuned_model_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [121]:
#use the finetuned model.
def correct_spelling_after_fine_tuning(word):
    input_ids = tokenizer.encode("correct:" + word, return_tensors="pt")

    # Correct spelling
    outputs = model_finetuned_spelling_en_1000_words.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

    # Decode the output
    correction = finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return correction

# Example text
word = "givie"
correction = correct_spelling_after_fine_tuning(word)
print("Correction:", correction)


Correction: general


In [137]:
%%time
log_message(f"############ run inference AFTER finetuning against SEEN data using LOCAL model")
for i, row in enumerate(seen_validation_data):
  corrected_text = correct_spelling_after_fine_tuning(row[0])
  print(f"{i}:{row[0]}--->{corrected_text} vs. {row[1]}")
  log_message(f"{i}:{row[0]}--->{corrected_text}")

0: tecnical--->technical vs. technical
1: scienct--->sci vs. science
2: agrement--->agreement vs. agreement
3: carachter--->car vs. character
4: reffered--->returned vs. referred
5: navir--->n vs. never
6:dutie--->due vs. duty
7:flwo--->financial vs. flow
8: finill--->financial vs. final
9: innyns--->in vs. in
10: nutural--->economic vs. natural
11: sehie--->place vs. see
12: tecnology--->technology vs. technology
13: volumee--->volume vs. volume
14: defindant--->source vs. defendant
15: kpep--->k vs. keep
16: Wesrt--->we vs. West
17: regiin--->rule vs. region
18:capacitiy--->capacity vs. capacity
19: dayz--->day vs. days
20: feildling--->way vs. field
21: kewe--->ke vs. knew
22: maede--->ma vs. made
23: devoloped--->developed vs. developed
24: basik--->base vs. basic
25: timse--->time vs. times
26:fijure--->financial vs. Figure
27: stnadarad--->s vs. standard
28: soin--->so vs. son
29: buisnesss--->cause vs. business
30: begnning--->asked vs. beginning
31: cease--->cease vs. case
32: 

In [133]:
%%time
log_message(f"############ run inference AFTER finetuning against UNSEEN data using LOCAL model")
for i, row in enumerate(unseen_validation_data):
  corrected_text = correct_spelling_after_fine_tuning(row[0])
  print(f"{i}:{row[0]}--->{corrected_text} vs. {row[1]}")
  log_message(f"{i}:{row[0]}--->{corrected_text}")

0:Appel--->appel vs. Apple
1:Booke--->Book vs. Book
2:Kare--->State vs. Car
3:Dög--->Dö vs. Dog
4:Hoise--->House vs. House
5:Trei--->State vs. Tree
6:Freind--->made vs. Friend
7:Hapy--->have vs. Happy
8:Bieg--->British vs. Big
9:Littel--->London vs. Little
10:Rune--->State vs. Run
11:Jummp--->June vs. Jump
12:Sée--->State vs. See
13:Heer--->He vs. Hear
14:Saiy--->State vs. Say
15:Goe--->Go vs. Go
16:Comey--->Com vs. Come
17:Stoip--->State vs. Stop
18:Starit--->start vs. Start
19:Fynish--->French vs. Finish
20:Wont--->made vs. Want
21:Neid--->State vs. Need
22:Likie--->State vs. Like
23:Lovve--->place vs. Love
24:Heelp--->He vs. Help
25:Tayk--->State vs. Take
26:Givv--->Georgia vs. Give
27:Mekke--->House vs. Make
28:Doe--->did vs. Do
29:Noow--->No vs. Know
30:Thinck--->Th vs. Think
31:Fiel--->State vs. Feel
32:Sée--->State vs. See
33:Heer--->He vs. Hear
34:Seee--->See vs. See
35:Heer--->He vs. Hear
36:Eit--->first vs. Eat
37:Drinik--->Dr vs. Drink
38:Sleeep--->length vs. Sleep
39:Wakie-